## The Math Behind the Models

### Markov Chain Monte Carlo

Our preferred method for determining the positions of putative TF binding sites, as well as to understand how mutations impact gene expression, is to perform Markov Chain Monte Carlo (MCMC) inference. MCMC gets its name from two processes, Monte Carlo and Markov Chain. Monte Carlo is a method for estimating features of a distribution by randomly drawing samples from the distribution. For example, one could estimate the mean or standard deviation of a distribution by drawing random samples and computing the mean and standard deviation for those samples. MCMC methods are often used when functions are not amenable to analytical solutions or calculations. MCMC methods allow the expectation value of a given parameter, and its uncertainty without requiring us to have full access to the underlying probability distribution. As with many cases in biology, the true underlying probability distribution is often complicated and difficult to access.

Our Reg-Seq method currently uses human intuition to determine binding sites (based on the characteristics of information footprints, discussed later in this Wiki protocol), but to validate our binding site choices, and to capture all regions of a sequence that are important for gene expression, we need to also computationally identify regions where gene expression is changed significantly up or down by mutation (p < 0.01), and discard any potential sites which do not fit this criteria. We infer the effect of mutation using MCMC and we use the distribution of parameters from the inference to form a 99 % confidence interval for the average effect of mutation across a 15 base pair region. We include binding sites that are statistically significant at the 0.01 level in any of the tested growth conditions.

One difficulty with estimating the mutual information from model predictions is that base pair identity A, C, G, T and the gene expression level µ are both discrete variables, while binding energy predictions from the model (x) is
a continuous variable. Formally, the mutual information is given by

\begin{align}
I(\mu, x) = \int_{- \infty}^{+ \infty} dx \sum_{u}^{} p(x, \mu) \log_{2} \left( \frac{p(x, \mu)}{p(x) p(\mu)} \right) \tag{1}
\end{align}

where µ is a measure of gene expression,

\begin{align}
\mu = \begin{cases}
  0, \quad \text{for sequencing reads from DNA library}\\[1em]
  1, \quad \text{for sequencing reads originating from mRNA} \tag{2}
\end{cases}
\end{align}

during Reg-Seq.

The probability distribution, $p$, is not one that we have full access to, as we only have a discrete set of predictions (one from each of the N unique DNA sequences in our data set). To compensate for the issues that can arise from estimating a continuous distribution from discrete data, we make use of the fact that any transformation that preserves the rank order (for instance multiplying all model predictions by a constant) the mutual information is unchanged.

We will define $z_q$ as the rank order in binding energy predictions of the $qth$ sequence. We estimate $I(\mu,z)$ by first calculating binding energy predictions

\begin{align}
x = \sum_{i=1}^L \sum_{j=A}^T \theta_{ij} \cdot \delta_{ij} ,\tag{3}
\end{align}

where $\delta_{ij}$ is the Kronecker-Delta and then converting them to a rank order predictions $z$.

We then discretize the energy predictions into 1000 "bins" and convolve with a Gaussian kernel to estimate the probability distribution $p(z, \mu)$. We can then calculate the mutual information with

\begin{align}
I(\mu,z)_{smoothed} = \sum_{z=1}^{1000} \sum{\mu} F(\mu,z) \log_2 \frac{F(\mu,z)}{F(z) \cdot F(\mu)} \tag{4}
\end{align}

where $F(\mu,z)$ is the probability distribution and $p(\mu,z)$ is estimated from finite data.

### Markov Chain Monte Carlo fitting procedure

As proven by [Justin Kinney (2008)](https://pdfs.semanticscholar.org/56ce/a3cb3609844a0df0554f99524dbb96479c2d.pdf) in his disseration, "Biophysical Models of Transcriptional Regulation from Sequence Data", the likelihood of a model that predicts gene output is

\begin{align}
L(\theta|{\mu_s}) \propto 2^{NI_{smooth}(\mu,z)} \tag{5}
\end{align}

where $N$ is the total number of independent sequences, $I_{smooth}$ is the smoothed mutual information between gene expression, $\mu$, and DNA sequence $z$.

The probability distributions are very difficult to handle analytically. The reason why we use MCMC is that you can estimate properties using the target probability distribution without needing to know the distribution. For example, we can estimate $\left\langle \theta \right \rangle$ by drawing many samples of $\theta$ using MCMC and taking the mean of the parameters.

We therefore need to construct a Markov Chain whose stationary distribution converges to the distribution of interest $p(\theta)$. A Markov chain is a sequence of values ${\theta_1, \theta_2,\theta_3,..., \theta_N}$ for $N$ steps. We can then find $\left \langle \theta \right \rangle$ with

\begin{align}
\left \langle \theta \right \rangle = \frac{\sum_{N=1}^{100} \theta_N}{N} \tag{6}
\end{align}

A Markov chain has no memory. That is the probability that the $N^{th}$ value in the chain takes a value $\theta_N$ depends only on the $N-1^{th}$ value in the chain. To make things a bit more concrete, let's leave aside $\theta$ for the time being. Imagine that we have a light switch, and we know the switch is "on" 25% of the time and "off" 75% of the time. For each "step" in our Markov chain, we can change the state of the switch; if the switch is on, we turn it off with some rate $k_{off}$, and if the switch is currently off, we turn it off with the rate $k_{on}$. A sequence of states will be generated. One example would be [off, off, off, on, on, on, off]. These states constitute a Markov chain and if the chain is continued for long enough, the stationary distribution will converge such that $p_{on}$ = 0.25 and $p_{off}$ = 0.75.

A Markov chain is stationary if detailed balance is satisfied between its states. The condition of detailed balance obtains if the total rate of transitions from on to off is the same as the total rate of transitions from off to on. Mathematically, this condition can be written as

\begin{align}
k_{on} \times p_{off} = k_{off} \times p_{on} \tag{7}
\end{align}

This equation allows us to calculate $k_{on}$ and $k_{off}$. It follows that 

\begin{align}
\frac{k_{on}}{k_{off}} = \frac{p_{on}}{p_{off}}  = \frac{1}{3} \tag{8}
\end{align}

As long as the ratio of transition rates is satisfied and enough steps are taken, then the stationary distribution will converge to the proper distribution.

For the far more complicated task of estimating $p(\theta)$, we can fall back on the standard Metroplis-Hastings sampling algorithm. $\theta$ will be a matrix where $\theta_{ij}$ will be the energetic contributions to the energy matrix for the $i^{th}$ position and the $j^{th}$ base pair where $i \in {1,2,3,...,L}$ and $j \in {A,C,G,T}$. We can then follow the procedure:

1. Start with a random energy matrix $\theta_0$.
2. Make a random perturbation $d \theta_0$ to $\theta_0$. This perturbation will have a small adjustment to each element of $\theta$.
3. Compute the model likelihoods $L(\theta_0)$ and $L(\theta_0 + d\theta$ using (Eqn. 5).
4. If $L(\theta_0)$ and $L(\theta_0 + d \theta > L(\theta$, accept the new parameter values $\theta_0$ + $d\theta$ as the next element in the Markov chain $\theta_1$. Otherwise, accept $\theta_0$ + $d \theta$ with probability $\frac{L(\theta_0 + d\theta)}{L(\theta_0)}$. If the step is rejected, the next element $\theta_1$ in the Markov chain is reset to its previous value $\theta_0$. The acceptance/rejection probabilities mean that detailed balance is satisfied between the states $\theta_0$ and $\theta_0 + d\theta$.
5. Repeat steps 2-4 of this procedure until the chain converges to the stationary distribution. In practice this can be determined by monitoring when the mutual information plateaus.
6. To be certain that the distribution has in fact converged to the proper stationary distribution, multiple Markov Chains should be run starting with step 1. If all the chains converge to the same distribution, then they have properly converged.

The end result of these model-fitting efforts is an optimized linear binding energy matrix. You can get a measure of the uncertainty in $\theta_{ij}$ by forming a confidence interval out of the distribution of parameters formed from the Markov Chain. This inference is performed using the MPAthic software.

Now that we have discussed MCMC, let's now turn to the other _potential_ method for data analysis: least squares analysis.

### Least Squares Analysis

In the Reg-Seq paper, we used exclusively MCMC analysis to interpret data emanating from sequencing experiments. However, it is still conceptually important to understand how Least Squares analysis works, as this offers an alternative method for analyzing the sequencing data.

Our least squares inference can be found in a paper by [Ireland and Kinney, _bioRxiv_](https://www.biorxiv.org/content/10.1101/054676v2).

Least squares provides a computationally simple inference procedure that overcomes the most onerous restrictions of enrichment ratio calculations. It can be used to infer any type of linear model, including both matrix models and neighbor models. It can also be used on data that consists of more than two bins. The idea behind the least squares approach is to
choose parameters $\theta^{LS}$ that minimize a quadratic loss function. Specifically, we use

\begin{align}
\theta^{LS} = \mathrm{argmin}_{\theta} L(\theta) \tag{9}
\end{align}

where

\begin{align}
L(\theta) = \sum_M \sum_{n | M} \frac{[r (S^n, \theta) - \mu_M]^2 }{\sigma_M^2} + \alpha \sum_i \theta_i^2 \tag{10}
\end{align}

Here, $\mu_M$ is the assumed mean activity of sequences in bin $M$, $\sigma_M^2$ is the assumed variance in the activities of such sequences, $i$ indexes all parameters in the model, and $\alpha$ is the "ridge regression" regularization parameter. By using the objective function $L(\theta)$, one can rapidly compute values of the optimal parameters $\theta$ using standard algorithms.

One downside to least squares inference is the need to assume specific values for $\mu_M$ and for $\sigma_M^2$ for each bin $M$. MPAthic allows the user to manually specify these values. There is a danger here, since assuming incorrect values for $\mu_M$ and $\sigma_M^2$ will generally lead to bias in the inferred parameters $\theta^{LS}$. In practice, however, the default choice of $\mu_M = M$ and $\sigma_M^2 = 1$ often works surprisingly well when bins are arranged from lowest to highest average activity. Another downside to least squares is the need to assume that experimental noise – specifically, $p(R|M)$ - is Gaussian. Only in such cases does least squares inference correspond to a meaningful maximum likelihood calculation. In massively parallel assays, however, noise is often strongly non-Gaussian. In such situations, least squares inference cannot be expected to yield correct model parameters for any choice of $\mu_M = M$ and $\sigma_M^2 = 1$.

Now that we have discussed both MCMC and least squares analysis, we now turn our attention to the _visualization_ of our analyzed data. Now that we have coaxed the sequencing data into a usable format -- and performed inference on the data -- we get to the fun part: plotting the data! In the [original Reg-Seq](https://www.biorxiv.org/content/10.1101/2020.01.18.910323v3) paper, we visualize sequencing data in three ways: information footprints, sequence logos and energy matrices. We discuss the _mathematics_ behind each of these visual formats next, and then conclude this Wiki protocol with in-depth details on how we plot these three visual tools.

## The Math Behind the Plots

### Mathematics of Information Footprints

This section can be found in Appendix 3 of the Reg-Seq paper.

### Mathematics of Sequence Logos

Sequence logos provide a simple way to visualize the sequence specificity of a transcription factor to DNA, as well as the amount of information present at each position. Here we describe how we generate them using either known genomic binding sites or the energy matrices from our Reg-Seq data. In each case we need to calculate a $4 \times L$ position weight matrix for a binding site of length $L$, which is used to estimate the position-dependent information content that will then be used to construct a sequence logo.

#### Generating Position Weight Matrices from Known Genomic Binding Sites.

To construct a position weight matrix using genomic binding sites, we must first align all the available binding site sequences and determine the nucleotide statistics at each position. Specifically, we count the number of each nucleotide, $N_{ij}$, at each position along the binding site. Here the subscript $i$ refers to the position, while $j$ refers to the nucleotide, A, C, G, or T. We can then calculate a position probability matrix (also $4 \times L$ where each entry is found by dividing these counts by the total number of sequences in our alignment,

\begin{align}
p_{ij} = \frac{N_{ij}}{N_g} \tag{24}
\end{align}

Note that in situations where the number of aligned sequences is small (e.g., less than five), we typically add 1 pseudocount sequence to regularize the probabilities of the counts in the calculation of position probabilities,

\begin{align}
p_{ij} = \frac{N_{ij} + B_p}{N_g + 4 \cdot B_p} \tag{25}
\end{align}

where $B_p$ is the value of the pseudocount. The argument for their use is that when selecting from a small number of binding site sequences, there is a chance that infrequent nucleotides will be absent, and assigning them a probability $p_{ij}$ of zero may be too stringent of a penalty. We let $B_p$ = 0.1. In the limit of zero binding site sequences (i..e with no sequences observed), this will result in probabilities $p_{ij}$ approximately equal to the background probability used in calculating the position weight matrix below (and a non-informative sequence logo).

Finally, the values of the position weight matrix are found by calculating the log probabilities relative to a background model,

\begin{align}
PW M_{ij} = log_2 \frac{p_{ij}}{b_j} \tag{26}
\end{align}

The background model reflects assumptions about the genomic background of the system under investigation. For instance, in many cases it may be reasonable to assume each base is equally likely to occur. Given that we know the base frequencies for _E. coli_, we choose a background model that reflects these frequencies $b_j$: A = 0.246, C = 0.254, G = 0.254, and T = 0.246 for strain MG1655; [BioNumbers
ID 100528](http://bionumbers.hms.harvard.edu). The value at the $i, j th$ position will be zero if the probability, $p_{ij}$, matches that of the background model, but non-zero otherwise. This reflects the fact that base frequencies matching the background model tell us nothing about the binding preferences of the transcription factor, while deviation from this background frequency indicates sequence specificity.

#### Generating Position Weight Matrices from Reg-Seq Data.

Next we construct a position weight matrix using our Reg-Seq data. Here, we appeal to the result from [Berg and von Hippel](https://www.ncbi.nlm.nih.gov/pubmed/3612791), that the logarithms of the base frequencies above should be proportional to their binding energy contributions. Berg and von Hippel considered a statistical mechanical system containing $L$ independent binding site positions, with the choice of nucleotide at each position corresponding to a change in the energy level by $\epsilon_{ij}$ relative to the lowest energy state at that position.

$\epsilon_{ij}$ corresponds to the energy entry from our energy matrix, scaled to absolute units, $\epsilon_{ij}$ = $A \dot \theta_{ij} \dot B$, where $\theta_{ij}$ is the $i, jth$ entry. An important assumption is that all nucleotide sequences that provide an equivalent binding energy will have equal probability of being present as a binding site. In this way, we can relate the binding energies considered here to the statistical distribution of binding sites in the previous section. The probability $p_{ij}$ of choosing a nucleotide at position $i$ will then be proportional to the probability that position $i$ has energy $\epsilon_{ij}$. Specifically, the probabilities will be given by their Boltzmann factors normalized by the sum of states for all nucleotides,

\begin{align}
p_{ij} = \frac{b_j \cdot e^{-\beta A \cdot \theta_{ij} \cdot s_{ij}}}{\sum_{j=A}^{T} b_j \cdot e^{-\beta A \cdot \theta_{ij} \cdot s_{ij}}} \tag{27}
\end{align}

where $\beta = 1 / k_B T$, $k_B$ is Boltzmann’s constant and $T$ is the absolute temperature. As above, $b_j$ refers to the background probabilities of each nucleotide. Note that the energy scaling factor B drops out of this equation since it is shared across each term. One difficulty that arises when we use energy matrices that are not in absolute energy units is that we are left with an unknown scale factor $A$, preventing calculation of $p_{ij}$. We appeal to the expectation that mismatches usually involve an energy cost of 1-3 $k_B T$. In other work within our group, we have
found this to be a reasonable assumption for _LacI_. Therefore, we approximate it such that the average
cost of a mutation is 2 $k_B T$.

#### Additional Details on Sequence Logos

With our position weight matrices in hand, we now construct sequence logos by calculating the average
information content at each position along the binding site. With our four letter alphabet there is a
maximum amount of information of 2 bits $[\log_2 4$ = 2 bits) at each position $i$. The information content
will be zero at a position when the nucleotide frequencies match the genomic background, and will have
a maximum of 2 bits only if a specific nucleotide is completely conserved. The total information content
at position $i$ is determined through calculation of the Shannon entropy, and is given by

\begin{align}
I_i = \sum_{j=A}^{T} p_{ij} \cdot \log_2 \frac{p_{ij}}{b_i} = \sum_{j=A}^{T} p_{ij} \cdot \mathrm{PWM}_{ij} \tag{28}
\end{align}

Here, $\mathrm{PWM}_{ij}$ refers to the $i, jth$ entry in the position weight matrix. The total information content contained in the position weight matrix is then the sum of information content across the length of the binding site.

To construct a sequence logo, the height of each letter at each position $i$ is determined by $\mathrm{SeqLogo}_{ij} = p{ij} \cdot I_i$, which is in units of bits. This causes each nucleotide in the sequence logo to be displayed as the proportion of the nucleotide expected at that position scaled by the amount of information contained at that position. To construct and plot sequence logos, we use custom Python code written by Justin Kinney; this code is discussed in a subsequent section of this protocol.